# A1111 Model Installer — **SDXL (1.0)**

Downloads SDXL models into your persistent A1111 data directory.

- **Data root:** `/workspace/a1111`
- **Model paths:** `/workspace/a1111/models/...`

### What’s covered (SDXL)
- **Base & Refiner checkpoints**
- **VAE** (for SDXL)
- **ControlNet XL** common models
- **AnimateDiff XL**, **ReActor**, or other extension models (optional)

### Notes
- Requires `huggingface_hub` for repo-based downloads (available in your base image).
- You can use your Hugging Face access token if any model is gated.
- Safe to rerun — existing files are skipped.
- Run the **Restart WebUI** cell at the end when finished.

In [ ]:
# --- Configuration ---
import os
from pathlib import Path

WEBUI_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
HF_TOKEN = os.environ.get("HF_TOKEN", "")  # optional HF auth token

INSTALL_CHECKPOINTS = True
INSTALL_VAES        = True
INSTALL_CONTROLNET  = True
INSTALL_OTHERS      = False  # AnimateDiff, ReActor, etc.

print("WEBUI_ROOT:", WEBUI_ROOT)
print("HF_TOKEN set:", bool(HF_TOKEN))

# Destination directories
CKPT_DIR = WEBUI_ROOT / "models" / "Stable-diffusion"
VAE_DIR  = WEBUI_ROOT / "models" / "VAE"
CN_DIR   = WEBUI_ROOT / "models" / "ControlNet"
OTH_DIR  = WEBUI_ROOT / "models" / "Extras"

for d in [CKPT_DIR, VAE_DIR, CN_DIR, OTH_DIR]:
    d.mkdir(parents=True, exist_ok=True)
CKPT_DIR, VAE_DIR, CN_DIR

## Model lists (SDXL)
You can uncomment or add your own links below.

- **Hugging Face**: specify `repo_id` and `filename` (preferred)
- **Direct**: raw HTTPS links (CivitAI, etc.)

In [ ]:
# --- Checkpoints (SDXL) ---
CHECKPOINTS_HF = [
    # Base model
    # {"repo_id": "stabilityai/stable-diffusion-xl-base-1.0", "filename": "sd_xl_base_1.0.safetensors"},
    # Refiner
    # {"repo_id": "stabilityai/stable-diffusion-xl-refiner-1.0", "filename": "sd_xl_refiner_1.0.safetensors"},
]
CHECKPOINTS_DIRECT = [
    # Optional: CivitAI or other .safetensors direct URLs
]

# --- VAEs ---
VAES_HF = [
    # Example: {"repo_id": "madebyollin/sdxl-vae-fp16-fix", "filename": "sdxl_vae.safetensors"},
]
VAES_DIRECT = []

# --- ControlNet XL ---
CONTROLNET_HF = [
    # {"repo_id": "diffusers/controlnet-sdxl-1.0", "filename": "diffusion_pytorch_model.safetensors", "dest": "controlnet-sdxl.safetensors"},
    # {"repo_id": "lllyasviel/controlnet-sdxl-1.0", "filename": "controlnet-canny-sdxl-1.0.safetensors"},
]
CONTROLNET_DIRECT = []

# --- Optional extension-related (AnimateDiff XL, ReActor, etc.) ---
OTHERS_DIRECT = [
    # AnimateDiff XL example
    # "https://huggingface.co/guoyww/animatediff/resolve/main/SDXL/sdxl_motion_module.safetensors",
]

print("Lists ready; edit or uncomment items you want to download.")

## Downloader core
Handles both HF and direct URLs, reusing the same logic as the SD1.5 installer.

In [ ]:
import os, shutil, requests
from pathlib import Path
from urllib.parse import urlparse

try:
    from huggingface_hub import hf_hub_download
    HF_OK = True
except Exception:
    HF_OK = False

def human(n):
    for unit in ['B','KB','MB','GB']:
        if n < 1024:
            return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url, dest):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return
    r = requests.get(url, stream=True)
    r.raise_for_status()
    size = int(r.headers.get('Content-Length', 0))
    print(f"→ {url} ({human(size)})")
    tmp = dest.with_suffix('.part')
    with open(tmp, 'wb') as f:
        for chunk in r.iter_content(1024*1024):
            f.write(chunk)
    tmp.replace(dest)
    print("✔ Saved:", dest)

def download_hf(repo_id, filename, dest_dir, dest_name=None):
    if not HF_OK:
        print("huggingface_hub not available; skipping")
        return
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest)
        return
    try:
        path = hf_hub_download(repo_id=repo_id, filename=filename, token=(HF_TOKEN or None))
        shutil.copy2(path, dest)
        print("✔ HF downloaded:", dest)
    except Exception as e:
        print("✖ HF error for", repo_id, filename, ":", e)

print("Downloader ready.")

## Run installers
Safe to rerun anytime; existing files are skipped.

In [ ]:
if INSTALL_CHECKPOINTS:
    print("\n=== SDXL Checkpoints ===")
    for it in CHECKPOINTS_HF:
        download_hf(it["repo_id"], it["filename"], CKPT_DIR, it.get("dest"))
    for u in CHECKPOINTS_DIRECT:
        download_direct(u, CKPT_DIR / Path(urlparse(u).path).name)

if INSTALL_VAES:
    print("\n=== SDXL VAEs ===")
    for it in VAES_HF:
        download_hf(it["repo_id"], it["filename"], VAE_DIR, it.get("dest"))
    for u in VAES_DIRECT:
        download_direct(u, VAE_DIR / Path(urlparse(u).path).name)

if INSTALL_CONTROLNET:
    print("\n=== ControlNet XL ===")
    for it in CONTROLNET_HF:
        download_hf(it["repo_id"], it["filename"], CN_DIR, it.get("dest"))
    for u in CONTROLNET_DIRECT:
        download_direct(u, CN_DIR / Path(urlparse(u).path).name)

if INSTALL_OTHERS:
    print("\n=== Optional extras (AnimateDiff XL, ReActor, etc.) ===")
    for u in OTHERS_DIRECT:
        download_direct(u, OTH_DIR / Path(urlparse(u).path).name)

print("\nAll done.")

## Restart WebUI

In [ ]:
import subprocess, shlex, time

print("Restarting A1111 (supervisord)…")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    print(out.stdout or out.stderr or "✔ Restart command sent.")
except Exception as e:
    print("Manual restart command if needed:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")